In [ ]:
%pylab ipympl
import scipy.integrate

import reaclib
import isotopes

In [ ]:
pp_r, pp_e, r_pp = reaclib.read_file("./reactions/p-p-d-ec")
d_r, d_e, r_d = reaclib.read_file("./reactions/d-pg-he3-de04")
h3_r, h3_e, r_h3 = reaclib.read_file("./reactions/he3-he3pp-he4-nacr")
print("Reactions:")
print(isotopes.reaction_to_str(pp_r))
print(isotopes.reaction_to_str(d_r))
print(isotopes.reaction_to_str(h3_r))

# X = [p, d, h3, a]
Y_0 = [0.97/1.0, 0.01/2, 0.01/3, 0.01/4]
rho = 1 # g/cm^3
T9 = 0.015

def dYdt(t, Y):
    p, d, h3, a= Y
    lambda1 = p**2*rho*r_pp(T9)/2
    lambda2 = d*p*rho*r_d(T9)
    lambda3 = h3**2*rho*r_h3(T9)/2
    dpdt = -2*lambda1 - lambda2 + 2*lambda3
    dddt = lambda1 - lambda2
    dh3dt = lambda2 - 2*lambda3
    dadt = lambda3
    return [dpdt, dddt, dh3dt, dadt]
lt_range = numpy.array([-5, 22])
lt = numpy.linspace(*lt_range, 1000)
t_range = 10.0**lt_range
t = 10.0**lt

solve = scipy.integrate.solve_ivp(dYdt, t_range, Y_0, t_eval=t, method="LSODA", rtol=1e-9, atol=1e-9)


In [ ]:
t = solve["t"]/(60*60*24*365)
p, d, h3, a = solve["y"]
print(p.shape)
ifig = 0
ifig += 1; pyplot.close(ifig); pyplot.figure(ifig)

pyplot.loglog(t, 1*p, label="$H_1$")
pyplot.plot(t, 2*d, label="$H_2$")
pyplot.plot(t, 3*h3, label="$He_3$")
pyplot.plot(t, 4*a, label="$He_4$")

pyplot.xlabel("t [yr]")
pyplot.ylabel("mass fraction")
pyplot.title("PP-Chain at fixed $T$, $\\rho$")
pyplot.ylim(1e-4, 1.5)
pyplot.legend()
pyplot.show(ifig)